# Data Wrangling OpenStreetMap

A xml osm file was downloaded for a selected area from OpenStreetMap.org. This report details the data auditing and cleaning performed on the raw dataset. After the data is cleaned, it is transformed and stored in a Sqlite database. With the stored dataset, the data and the chosen area are further explored.

## Map

## Data Auditing

After auditing the street and postal code data, I found the following issues with the data source. The las_vegas_audit.py script is used to audit and find below issues. 
1. Various street name abbreviations for Avenue, Road, Boulevard, Parkway and Street are present. 
2. Some of the street names have numbers at the end of the name, although this issue is small. For Ex., '1': set(['Spanish Ridge Ave., Suite 1']
3. Some postal codes have more than 5 digits and some have alphabets at the begenning. Except one, all postal codes start with 89, which are the starting digits of postal codes in Las vegas.

Below mapping dictionary is created to change the abbreviated street names. update_street_name function is used to clean street names.

First five digits in postal code is extracted to clean the postal codes. update_postal_code function is used to extract first five digits of the postal code.

## Process XML elements and create csv

After cleaning street names and postal codes, XML elements are processed using las_vegas_csv.py script and below csv files are created. Using csv_db.py, data base tables have been created from csv files.

    las_vegas.osm ----> 215.7 MB
    las_vegas.db -----> 150 MB
    nodes.csv --------> 80.8 MB
    nodes_tags.csv ---> 2.9 MB
    ways.csv ---------> 7.9 MB
    ways_nodes.csv ---> 27.5 MB
    ways_tags.csv ----> 13 MB

## Data exploration

### Number of nodes

```sql
SELECT COUNT(*) FROM nodes;
```

### Number of ways

```sql
SELECT COUNT(*) FROM ways;
```

### Number of distinct users

```sql 
SELECT count(distinct uid) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) users; 
```

### Top 10 contributers

```sql
SELECT uid,count(*) AS num_contributions
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) a
GROUP BY 1
ORDER BY num_contributions DESC
LIMIT 10;
```

## Street names with numbers in the end

We can see below that we have street names with numbers in them. These street names should be cleaned in order to improve the data quality.

```sql
SELECT value
FROM nodes_tags
WHERE key='street' AND value REGEXP '[[:digit:]]$'
GROUP BY 1
LIMIT 10;
```

### Top Cities and Provinces

```sql
SELECT value,count(*) as num
FROM
(
  SELECT value FROM nodes_tags WHERE key='city' or key='province'
  UNION ALL
  SELECT value FROM ways_tags WHERE key='city' or key='province'
)
GROUP BY 1
ORDER BY 2;
```

### Top amenities

```sql
SELECT value,count(*)
FROM nodes_tags
WHERE key='amenity'
GROUP BY 1
ORDER BY count(*) DESC
LIMIT 20;
```

I'm surprised to not see hotels in the top list of amenities in Vegas. So, I dig deeper into the top keys.

### Count of keys

```sql
SELECT key,count(*)
FROM nodes_tags
GROUP BY 1
ORDER BY count(*) DESC
LIMIT 20;
```

I don't see any other key that might give information about amenities. So, lets check top streets and postal codes.

### Top streets and postal codes

```sql
SELECT value,count(*)
FROM nodes_tags
WHERE key='street'
GROUP BY 1
ORDER BY count(*) DESC
LIMIT 20;
```

We see some famous streets in the list.

```sql
SELECT value,count(*)
FROM nodes_tags
WHERE key='postcode'
GROUP BY 1
ORDER BY count(*) DESC
LIMIT 20;
```

### Top 10 cuisines

```sql
SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;
```

## Conclusion

Las Vegas dataset has been actively contributed by users. However, the dataset has a few data quality issues such as typos in street names, city & province names and postal codes.  

These typos could be reduced by implementing country and city specific rules. For example, users should not be allowed to enter postal codes more than five digits and it should start from 88 or 89 for Las Vegas. Another way to reduce typos could be to suggest correct city or street names in case a user is typing wrong spelling. However, these solutions should be quick to adapt to future changes in the street names and postal codes. This requires a great deal of effort from OpenStreetMap community when done in a world scale. If the community is not quick to adapt the content, it could block users from entering current information and would discourage them from future contribution.

As a further work, the Las Vegas map could be improved with information of famous, tourist locations. In order to clean the dataset further, regular expressions could be used to clean street names that have numbers at the end.